<cell_type>markdown</cell_type># 协调器-工作者工作流程

## 简介

你是否曾经需要对同一任务有多个视角，但无法预测哪些视角最有价值？协调器-工作者模式通过让一个中心 LLM 分析每个独特任务并动态确定最适合委托给专业工作者 LLM 的子任务来解决这个问题。

传统方法要么需要多次手动提示，要么使用硬编码的并行化，无论上下文如何都生成相同的变体。

使用这种方法，协调器 LLM 分析任务，确定哪些变体对这个特定案例最有价值，然后委托给工作者 LLM 生成每个变体。

### 你将要构建的系统

一个接受产品描述请求的系统：

1. 分析哪种类型的营销文案有价值
2. 动态为工作者生成专门的任务描述
3. 产生针对不同受众优化的多种内容变体
4. 返回来自所有工作者的协调结果

### 先决条件

- Python 3.9 或更高版本
- 设置 Anthropic API 密钥作为环境变量：`export ANTHROPIC_API_KEY='your-key'`
- 基本理解提示工程
- 熟悉 Python 类和类型提示


### 何时使用此工作流程

此工作流程非常适合无法预先预测所需子任务的复杂任务。与简单并行化的关键区别在于其灵活性——子任务不是预定义的，而是由协调器根据特定输入确定的。

**在以下情况下使用此模式：**

- 任务需要多个不同的方法或视角
- 最优子任务取决于特定输入
- 需要比较不同的策略或风格

**在以下情况下不要使用此模式：**

- 简单、单输出任务（不必要的复杂性）
- 延迟关键（多个 LLM 调用增加开销）
- 子任务可预测且可以预定义（使用更简单的并行化）

## 工作原理

协调器-工作者模式分两个阶段运行：

1. **分析与规划阶段**：协调器 LLM 接收任务和上下文，分析哪种方法有价值，并以 XML 格式生成结构化的子任务描述。

2. **执行阶段**：每个工作者 LLM 接收：
   - 原始任务作为上下文
   - 其特定的子任务类型和描述
   - 提供的任何额外上下文

协调器在运行时决定创建什么子任务，使其比预定义的并行工作流程更具适应性。

<cell_type>markdown</cell_type>## 设置

### 安装
```bash
pip install anthropic
```

### 辅助函数
此示例使用来自 `util.py` 的辅助函数来进行 LLM 调用和解析 XML 响应：

- `llm_call(prompt, system_prompt="", model="claude-sonnet-4-5")`：向 Claude 发送提示并返回文本响应
- `extract_xml(text, tag)`：使用正则表达式从 XML 标签中提取内容

这些实用程序处理 API 认证（从环境读取 `ANTHROPIC_API_KEY`）并为协调器-工作者模式提供简单接口。您可以在 [util.py](util.py) 中查看完整的实现。

## 实现

`FlexibleOrchestrator` 类协调两阶段工作流程：

**关键设计决策：**
- 提示是接受运行时变量（`task`, `context`）的模板，以实现灵活性
- XML 用于结构化输出解析（可靠且适合语言模型的格式）
- 工作者接收原始任务和他们的特定指令，以获得更好的上下文
- 错误处理验证工作者返回非空响应

实现包括：
- `parse_tasks()`：将协调器的 XML 输出解析为结构化任务字典
- `FlexibleOrchestrator.process()`：调用协调器，然后调用工作者的主要协调逻辑
- 响应验证以捕获和处理空工作者输出

In [8]:
from typing import Dict, List, Optional
from util import llm_call, extract_xml

# Model configuration
MODEL = "claude-sonnet-4-5"  # Fast, capable model for both orchestrator and workers


def parse_tasks(tasks_xml: str) -> List[Dict]:
    """Parse XML tasks into a list of task dictionaries."""
    tasks = []
    current_task = {}

    for line in tasks_xml.split("\n"):
        line = line.strip()
        if not line:
            continue

        if line.startswith("<task>"):
            current_task = {}
        elif line.startswith("<type>"):
            current_task["type"] = line[6:-7].strip()
        elif line.startswith("<description>"):
            current_task["description"] = line[12:-13].strip()
        elif line.startswith("</task>"):
            if "description" in current_task:
                if "type" not in current_task:
                    current_task["type"] = "default"
                tasks.append(current_task)

    return tasks


class FlexibleOrchestrator:
    """Break down tasks and run them in parallel using worker LLMs."""

    def __init__(
        self,
        orchestrator_prompt: str,
        worker_prompt: str,
        model: str = MODEL,
    ):
        """Initialize with prompt templates and model selection."""
        self.orchestrator_prompt = orchestrator_prompt
        self.worker_prompt = worker_prompt
        self.model = model

    def _format_prompt(self, template: str, **kwargs) -> str:
        """Format a prompt template with variables."""
        try:
            return template.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Missing required prompt variable: {e}")

    def process(self, task: str, context: Optional[Dict] = None) -> Dict:
        """Process task by breaking it down and running subtasks in parallel."""
        context = context or {}

        # Step 1: Get orchestrator response
        orchestrator_input = self._format_prompt(self.orchestrator_prompt, task=task, **context)
        orchestrator_response = llm_call(orchestrator_input, model=self.model)

        # Parse orchestrator response
        analysis = extract_xml(orchestrator_response, "analysis")
        tasks_xml = extract_xml(orchestrator_response, "tasks")
        tasks = parse_tasks(tasks_xml)

        print("\n" + "=" * 80)
        print("ORCHESTRATOR ANALYSIS")
        print("=" * 80)
        print(f"\n{analysis}\n")

        print("\n" + "=" * 80)
        print(f"IDENTIFIED {len(tasks)} APPROACHES")
        print("=" * 80)
        for i, task_info in enumerate(tasks, 1):
            print(f"\n{i}. {task_info['type'].upper()}")
            print(f"   {task_info['description']}")

        print("\n" + "=" * 80)
        print("GENERATING CONTENT")
        print("=" * 80 + "\n")

        # Step 2: Process each task
        worker_results = []
        for i, task_info in enumerate(tasks, 1):
            print(f"[{i}/{len(tasks)}] Processing: {task_info['type']}...")

            worker_input = self._format_prompt(
                self.worker_prompt,
                original_task=task,
                task_type=task_info["type"],
                task_description=task_info["description"],
                **context,
            )

            worker_response = llm_call(worker_input, model=self.model)
            worker_content = extract_xml(worker_response, "response")

            # Validate worker response - handle empty outputs
            if not worker_content or not worker_content.strip():
                print(f"⚠️  Warning: Worker '{task_info['type']}' returned no content")
                worker_content = f"[Error: Worker '{task_info['type']}' failed to generate content]"

            worker_results.append(
                {
                    "type": task_info["type"],
                    "description": task_info["description"],
                    "result": worker_content,
                }
            )

        # Display results
        print("\n" + "=" * 80)
        print("RESULTS")
        print("=" * 80)
        for i, result in enumerate(worker_results, 1):
            print(f"\n{'-' * 80}")
            print(f"Approach {i}: {result['type'].upper()}")
            print(f"{'-' * 80}")
            print(f"\n{result['result']}\n")

        return {
            "analysis": analysis,
            "worker_results": worker_results,
        }

<cell_type>markdown</cell_type>## 示例用例：营销变体生成

现在让我们通过一个实际示例来看看协调器-工作者模式的实际应用：为产品生成多种风格的营销文案。

**为什么这个示例很好地展示了模式：**
- 不同产品受益于不同的营销角度
- "最佳"变体取决于特定的产品功能和目标受众
- 协调器可以根据输入调整其策略，而不是使用固定模板

**提示设计说明：**
- 协调器提示要求 2-3 种方法，并提供 XML 结构指导
- 工作者提示为工作者提供完整上下文（原始任务、其风格和指导方针）
- 两个提示都使用清晰的 XML 格式以确保可靠解析

In [9]:
ORCHESTRATOR_PROMPT = """
Analyze this task and break it down into 2-3 distinct approaches:

Task: {task}

Return your response in this format:

<analysis>
Explain your understanding of the task and which variations would be valuable.
Focus on how each approach serves different aspects of the task.
</analysis>

<tasks>
    <task>
    <type>formal</type>
    <description>Write a precise, technical version that emphasizes specifications</description>
    </task>
    <task>
    <type>conversational</type>
    <description>Write an engaging, friendly version that connects with readers</description>
    </task>
</tasks>
"""

WORKER_PROMPT = """
Generate content based on:
Task: {original_task}
Style: {task_type}
Guidelines: {task_description}

Return your response in this format:

<response>
Your content here, maintaining the specified style and fully addressing requirements.
</response>
"""


orchestrator = FlexibleOrchestrator(
    orchestrator_prompt=ORCHESTRATOR_PROMPT,
    worker_prompt=WORKER_PROMPT,
)

results = orchestrator.process(
    task="Write a product description for a new eco-friendly water bottle",
    context={
        "target_audience": "environmentally conscious millennials",
        "key_features": ["plastic-free", "insulated", "lifetime warranty"],
    },
)


ORCHESTRATOR ANALYSIS


This task requires creating marketing copy for an eco-friendly water bottle. The core challenge is balancing product information with persuasive messaging while highlighting the environmental benefits. Different approaches would serve distinct marketing channels and audience segments:

1. A **feature-focused technical approach** would appeal to detail-oriented consumers who make purchasing decisions based on specifications, materials, and measurable environmental impact. This serves e-commerce listings and comparison shopping.

2. A **lifestyle-oriented emotional approach** would connect with values-driven consumers through storytelling and aspirational messaging, emphasizing how the product fits into an eco-conscious lifestyle. This serves social media and brand-building content.

3. A **benefit-driven practical approach** would focus on solving everyday problems while weaving in sustainability advantages, appealing to mainstream consumers who want both functi

<cell_type>markdown</cell_type>## 总结

您现在已经实现了一个协调器-工作者模式，该模式根据特定输入动态调整其任务分解。此模式生成了多种营销文案变体——每个都针对不同的受众和上下文量身定制——而无需您预定义这些变体应该是什么。

### 关键要点

**模式优势：**
- **适应性**：协调器为每个独特输入确定最佳方法
- **灵活性**：通过更改提示轻松应用于不同领域
- **结构化协调**：基于 XML 的通信确保可靠解析
- **错误恢复能力**：验证捕获并处理工作者失败

**此模式擅长的情况：**
- 需要多视角的内容生成（营销、文档、创意写作）
- 受益于不同分析视角的分析任务
- 分解策略取决于问题的解决问题

### 限制与考虑因素

**成本与延迟：**
- 需要 N+1 次 LLM 调用（1 个协调器 + N 个工作者）
- 此实现中的顺序处理（工作者一次运行一个）
- 为了更好的性能，考虑使用 `asyncio` 或线程池并行化工作者调用

**何时不使用此模式：**
- 具有单一、清晰输出的简单任务（增加的复杂性不合理）
- 延迟关键应用程序（多次 API 调用增加开销）
- 子任务始终相同的任务（改为使用预定义的并行化）

**需要考虑的失败模式：**
- 协调器可能没有最优地分解任务（提示工程至关重要）
- 工作者可能返回空或格式错误的响应（我们使用验证处理这个问题）
- 如果模型不完全遵循格式，XML 解析可能会失败（考虑使用 JSON 作为替代方案）

### 后续步骤

**增强此实现：**
1. 使用 `asyncio` 添加并行工作者执行以获得更好性能
2. 为失败的工作者实现重试逻辑
3. 添加综合阶段，让 LLM 合并工作者输出
4. 尝试不同的协调器策略（例如，要求更多/更少的子任务）

**适应您的用例：**
- 修改协调器提示以指导任务分解适应您的领域
- 调整工作者提示以提供领域特定指令
- 添加与您的应用程序相关的上下文参数
- 考虑为协调器使用 Claude Opus，为工作者使用 Claude Haiku 以优化成本与质量